<a href="https://colab.research.google.com/github/RodNSS/banned_book_exploration/blob/main/banned_prison_topics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 72.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 89.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   

In [2]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer, util
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from bertopic.representation import KeyBERTInspired
from bertopic.representation import MaximalMarginalRelevance
from bertopic.representation import ZeroShotClassification

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
prison = pd.read_csv('prison_topics_use2.csv')
prison

,Unnamed: 0,publication,author,date,year,month,day,reason,state_arc,publication_date,topic_description,author_2,publisher,printType,categories,State_Count,Topic
0,0,[ARABIC LANGUAGE CHARACTERS].,NaN,2019-12-18,2019.0,12.0,18.0,NaN,FL,1806,NaN,['Ahmad ibn Abou Bakr ibn Wahschih'],NaN,BOOK,NaN,20202,NaN
1,1,[ARABIC LANGUAGE CHARACTERS]. TAJWEED QURAN.,NaN,2021-09-09,2021.0,9.0,9.0,NaN,FL,NaN,NaN,NaN,NaN,NaN,NaN,20202,NaN
2,2,[KOREAN LANGUAGE CHARACTERS] HOLY SPIRIT LOVE.,NaN,2021-02-25,2021.0,2.0,25.0,NaN,FL,NaN,NaN,NaN,NaN,NaN,NaN,20202,NaN
3,3,[KOREAN LANGUAGE CHARACTERS].,NaN,2021-02-25,2021.0,2.0,25.0,NaN,FL,2021-02-10,Are you studying Korean and want to learn how ...,['Lilas Lingvo'],Independently Published,BOOK,NaN,20202,vocabulary_arabic_pronunciation_kanji
4,4,[KOREAN LANGUAGE CHARACTERS].,NaN,2021-02-25,2021.0,2.0,25.0,NaN,FL,2021-02-10,Are you studying Korean and want to learn how ...,['Lilas Lingvo'],Independently Published,BOOK,NaN,20202,vocabulary_arabic_pronunciation_kanji
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55273,55273,Mentalism Companion,NaN,2021-08-25,2021.0,8.0,25.0,NaN,RI,NaN,NaN,NaN,NaN,NaN,NaN,68,NaN
55274,55274,"You Can Read Anyone, Rules of the Game",NaN,2021-09-22,2021.0,9.0,22.0,NaN,RI,NaN,NaN,NaN,NaN,NaN,NaN,68,NaN
55275,55275,Sex Slave never judge a Women by her desires,NaN,2021-09-30,2021.0,9.0,30.0,NaN,RI,NaN,NaN,NaN,NaN,NaN,NaN,68,NaN
55276,55276,Moorish American Civics Guide,NaN,2021-09-30,2021.0,9.0,30.0,NaN,RI,2014-12-22,This civics guide is designed for those who la...,['Sheik Way-El'],Lulu.com,BOOK,NaN,68,blacks_civil_rights_movement


In [19]:
import calendar
import numpy as np

# Define a function to convert month name to its corresponding numerical digit
def convert_month_to_number(month_name):
    month_number = str(list(calendar.month_abbr).index(month_name.capitalize()))
    return month_number.zfill(2)  # Zero-pad the month number if needed

# Convert the date column for state_arc = 'IA'
prison.loc[prison['state_arc'] == 'IA', 'date'] = prison.loc[
    (prison['state_arc'] == 'IA') & (~prison['date'].isna()), 'date'].apply(
        lambda date: f"20{date[-2:]}-{convert_month_to_number(date[:-3])}"
) 

# Print the resulting prison dataframe
prison


,Unnamed: 0,publication,author,date,year,month,day,reason,state_arc,publication_date,topic_description,author_2,publisher,printType,categories,State_Count,Topic
0,0,[ARABIC LANGUAGE CHARACTERS].,NaN,2019-12-18,2019.0,12.0,18.0,NaN,FL,1806,NaN,['Ahmad ibn Abou Bakr ibn Wahschih'],NaN,BOOK,NaN,20202,NaN
1,1,[ARABIC LANGUAGE CHARACTERS]. TAJWEED QURAN.,NaN,2021-09-09,2021.0,9.0,9.0,NaN,FL,NaN,NaN,NaN,NaN,NaN,NaN,20202,NaN
2,2,[KOREAN LANGUAGE CHARACTERS] HOLY SPIRIT LOVE.,NaN,2021-02-25,2021.0,2.0,25.0,NaN,FL,NaN,NaN,NaN,NaN,NaN,NaN,20202,NaN
3,3,[KOREAN LANGUAGE CHARACTERS].,NaN,2021-02-25,2021.0,2.0,25.0,NaN,FL,2021-02-10,Are you studying Korean and want to learn how ...,['Lilas Lingvo'],Independently Published,BOOK,NaN,20202,vocabulary_arabic_pronunciation_kanji
4,4,[KOREAN LANGUAGE CHARACTERS].,NaN,2021-02-25,2021.0,2.0,25.0,NaN,FL,2021-02-10,Are you studying Korean and want to learn how ...,['Lilas Lingvo'],Independently Published,BOOK,NaN,20202,vocabulary_arabic_pronunciation_kanji
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55273,55273,Mentalism Companion,NaN,2021-08-25,2021.0,8.0,25.0,NaN,RI,NaN,NaN,NaN,NaN,NaN,NaN,68,NaN
55274,55274,"You Can Read Anyone, Rules of the Game",NaN,2021-09-22,2021.0,9.0,22.0,NaN,RI,NaN,NaN,NaN,NaN,NaN,NaN,68,NaN
55275,55275,Sex Slave never judge a Women by her desires,NaN,2021-09-30,2021.0,9.0,30.0,NaN,RI,NaN,NaN,NaN,NaN,NaN,NaN,68,NaN
55276,55276,Moorish American Civics Guide,NaN,2021-09-30,2021.0,9.0,30.0,NaN,RI,2014-12-22,This civics guide is designed for those who la...,['Sheik Way-El'],Lulu.com,BOOK,NaN,68,blacks_civil_rights_movement


In [20]:
prison[prison['state_arc']=="IA"]

,Unnamed: 0,publication,author,date,year,month,day,reason,state_arc,publication_date,topic_description,author_2,publisher,printType,categories,State_Count,Topic
25712,25712,"A DECLARATION, A CALL TO GET ORGANIZED NOW FOR...",NaN,2021-05,2021.0,5.0,NaN,F1,IA,NaN,NaN,NaN,NaN,NaN,NaN,431,NaN
25713,25713,A GANGSTER'S CODE 3 - Bloody Betrayal,NaN,2021-06,2021.0,6.0,NaN,F1/F10,IA,NaN,NaN,NaN,NaN,NaN,NaN,431,NaN
25714,25714,A VENOM IN THE BLOOD (Hoffmann),NaN,2020-02,2020.0,2.0,NaN,F2/F6/F10,IA,NaN,NaN,NaN,NaN,NaN,NaN,431,NaN
25715,25715,ALL NATURAL BEAUTIES,NaN,2021-10,2021.0,10.0,NaN,F10,IA,2020-07-13,NaN,['Adam Koons'],NaN,BOOK,['Art'],431,NaN
25716,25716,ALVARADO'S PIN-UP NUDES - 2nd Edition,NaN,2020-07,2020.0,7.0,NaN,F10,IA,2019-09-28,Since Alvardo's Pin-Up Nudes was first publish...,['Robert Alvarado'],Schiffer Publishing,BOOK,['Photography'],431,bettie_collection_yeager_gil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26138,26138,"WORLD'S WORST CRIMES, THE - An A-Z of Evil Deeds",NaN,2021-07,2021.0,7.0,NaN,F1,IA,NaN,NaN,NaN,NaN,NaN,NaN,431,NaN
26139,26139,WRESTLER,NaN,NaN,NaN,NaN,NaN,NaN,IA,1992-08-03,The Wrestler's Body tells the story of a way o...,['Joseph S. Alter'],Univ of California Press,BOOK,['Social Science'],431,strength_nutrition_workouts_equipment
26140,26140,"Jan/Feb/Mar/Apr/May/Jun/July/Aug/Sep/Oct.01, #...",NaN,NaN,NaN,NaN,NaN,NaN,IA,NaN,NaN,NaN,NaN,NaN,NaN,431,NaN
26141,26141,YOUNG EXOTIC BEAUTIES,NaN,2019-08,2019.0,8.0,NaN,F10,IA,NaN,NaN,NaN,NaN,NaN,NaN,431,NaN


In [21]:
prison_filtered = prison.dropna(subset=['topic_description'])
prison_filtered


,Unnamed: 0,publication,author,date,year,month,day,reason,state_arc,publication_date,topic_description,author_2,publisher,printType,categories,State_Count,Topic
3,3,[KOREAN LANGUAGE CHARACTERS].,NaN,2021-02-25,2021.0,2.0,25.0,NaN,FL,2021-02-10,Are you studying Korean and want to learn how ...,['Lilas Lingvo'],Independently Published,BOOK,NaN,20202,vocabulary_arabic_pronunciation_kanji
4,4,[KOREAN LANGUAGE CHARACTERS].,NaN,2021-02-25,2021.0,2.0,25.0,NaN,FL,2021-02-10,Are you studying Korean and want to learn how ...,['Lilas Lingvo'],Independently Published,BOOK,NaN,20202,vocabulary_arabic_pronunciation_kanji
6,6,1 HOUR TRADE.,NaN,2019-07-31,2019.0,7.0,31.0,NaN,FL,2022-05-02T22:59:00Z,Please note: This is a companion version & not...,"['Everest Media,']",Everest Media LLC,BOOK,['Business & Economics'],20202,stocks_investment_forex_traders
10,10,10 WORST PSYCHOPATHS: THE MOST DEPRAVED KILLER...,NaN,2017-06-01,2017.0,6.0,1.0,NaN,FL,2016-10-15,"Often, the term 'psychopath' tends to be equat...",['Victor McQueen'],Arcturus Editions,BOOK,['Murderers'],20202,serial_bundy_case_murders
11,11,100 ANIMALS ADULT COLORING BOOK.,NaN,2019-08-28,2019.0,8.0,28.0,NaN,FL,2019-12-16,Animal Lovers Coloring Book with 100 Gorgeous ...,['A Design Creation'],NaN,BOOK,NaN,20202,coloring_stress_designs_pages
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55261,55261,Serial Killers,NaN,2021-06-09,2021.0,6.0,9.0,NaN,RI,2012-11-27,Bestselling true-crime writer Harold Schechter...,['Harold Schechter'],Simon and Schuster,BOOK,['True Crime'],68,serial_bundy_case_murders
55262,55262,Evil Women Deadlier than the male,NaN,2021-07-01,2021.0,7.0,1.0,NaN,RI,2012-07-13,Among the thirty-four cases covered are: Pamel...,['John Marlowe'],Arcturus Publishing,BOOK,['True Crime'],68,nursing_prescription_monographs_interactions
55270,55270,Ted Bundy- Conversations with a killer,NaN,2021-08-26,2021.0,8.0,26.0,NaN,RI,2000,Drawn from more than 150 hours of exclusive ta...,"['Stephen G. Michaud', 'Hugh Aynesworth']",Authorlink,BOOK,['Biography & Autobiography'],68,serial_bundy_case_murders
55271,55271,Fentanyl,NaN,2021-08-26,2021.0,8.0,26.0,NaN,RI,2020-03-14,Is there a way to end North America’s opioid e...,['Brodie Ramin'],Dundurn,BOOK,['Medical'],68,clinical_neurology_practice_adhd


In [22]:
# create copy for later merging
prison_filtered_copy = prison_filtered.copy()

In [23]:
prison_filtered_copy.drop_duplicates(subset='topic_description', keep='first', inplace=True)
prison_filtered_copy

,Unnamed: 0,publication,author,date,year,month,day,reason,state_arc,publication_date,topic_description,author_2,publisher,printType,categories,State_Count,Topic
3,3,[KOREAN LANGUAGE CHARACTERS].,NaN,2021-02-25,2021.0,2.0,25.0,NaN,FL,2021-02-10,Are you studying Korean and want to learn how ...,['Lilas Lingvo'],Independently Published,BOOK,NaN,20202,vocabulary_arabic_pronunciation_kanji
6,6,1 HOUR TRADE.,NaN,2019-07-31,2019.0,7.0,31.0,NaN,FL,2022-05-02T22:59:00Z,Please note: This is a companion version & not...,"['Everest Media,']",Everest Media LLC,BOOK,['Business & Economics'],20202,stocks_investment_forex_traders
10,10,10 WORST PSYCHOPATHS: THE MOST DEPRAVED KILLER...,NaN,2017-06-01,2017.0,6.0,1.0,NaN,FL,2016-10-15,"Often, the term 'psychopath' tends to be equat...",['Victor McQueen'],Arcturus Editions,BOOK,['Murderers'],20202,serial_bundy_case_murders
11,11,100 ANIMALS ADULT COLORING BOOK.,NaN,2019-08-28,2019.0,8.0,28.0,NaN,FL,2019-12-16,Animal Lovers Coloring Book with 100 Gorgeous ...,['A Design Creation'],NaN,BOOK,NaN,20202,coloring_stress_designs_pages
13,13,100 BEST SOLITAIRE GAMES.,NaN,2013-07-17,2013.0,7.0,17.0,NaN,FL,2013-09-01,table { }.font5 { color: black; font-size: 8pt...,NaN,Cardoza Publishing,BOOK,['Games'],20202,poker_play_em_limit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55256,55256,The Negro Question- Who am I,NaN,2021-04-29,2021.0,4.0,29.0,NaN,RI,2012-01-06,Finally a book filled with archaeological evid...,['Lee Cummings'],NaN,BOOK,NaN,68,blacks_civil_rights_movement
55259,55259,The Negro Question Part 4 - The Missing Link,NaN,2021-04-29,2021.0,4.0,29.0,NaN,RI,2015-03-05,This book is about the three black Kingdoms of...,['Lee Cummings'],Createspace Independent Publishing Platform,BOOK,['Black people'],68,blacks_civil_rights_movement
55262,55262,Evil Women Deadlier than the male,NaN,2021-07-01,2021.0,7.0,1.0,NaN,RI,2012-07-13,Among the thirty-four cases covered are: Pamel...,['John Marlowe'],Arcturus Publishing,BOOK,['True Crime'],68,nursing_prescription_monographs_interactions
55271,55271,Fentanyl,NaN,2021-08-26,2021.0,8.0,26.0,NaN,RI,2020-03-14,Is there a way to end North America’s opioid e...,['Brodie Ramin'],Dundurn,BOOK,['Medical'],68,clinical_neurology_practice_adhd


In [24]:
# create df with NaN topic_descriptions to merge back together after topics are identified from rest of data
df_with_nan = prison[pd.isnull(prison['topic_description'])]
df_with_nan

,Unnamed: 0,publication,author,date,year,month,day,reason,state_arc,publication_date,topic_description,author_2,publisher,printType,categories,State_Count,Topic
0,0,[ARABIC LANGUAGE CHARACTERS].,NaN,2019-12-18,2019.0,12.0,18.0,NaN,FL,1806,NaN,['Ahmad ibn Abou Bakr ibn Wahschih'],NaN,BOOK,NaN,20202,NaN
1,1,[ARABIC LANGUAGE CHARACTERS]. TAJWEED QURAN.,NaN,2021-09-09,2021.0,9.0,9.0,NaN,FL,NaN,NaN,NaN,NaN,NaN,NaN,20202,NaN
2,2,[KOREAN LANGUAGE CHARACTERS] HOLY SPIRIT LOVE.,NaN,2021-02-25,2021.0,2.0,25.0,NaN,FL,NaN,NaN,NaN,NaN,NaN,NaN,20202,NaN
5,5,[SERBIAN LANGUAGE CHARACTERS]. KAPATHOPTHE.,NaN,2021-07-29,2021.0,7.0,29.0,NaN,FL,NaN,NaN,NaN,NaN,NaN,NaN,20202,NaN
7,7,10 DAI THOU TUONG TRUNG QUAC.,NaN,2017-07-06,2017.0,7.0,6.0,NaN,FL,NaN,NaN,NaN,NaN,NaN,NaN,20202,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55272,55272,The Pocket Samurai,NaN,2021-08-26,2021.0,8.0,26.0,NaN,RI,NaN,NaN,NaN,NaN,NaN,NaN,68,NaN
55273,55273,Mentalism Companion,NaN,2021-08-25,2021.0,8.0,25.0,NaN,RI,NaN,NaN,NaN,NaN,NaN,NaN,68,NaN
55274,55274,"You Can Read Anyone, Rules of the Game",NaN,2021-09-22,2021.0,9.0,22.0,NaN,RI,NaN,NaN,NaN,NaN,NaN,NaN,68,NaN
55275,55275,Sex Slave never judge a Women by her desires,NaN,2021-09-30,2021.0,9.0,30.0,NaN,RI,NaN,NaN,NaN,NaN,NaN,NaN,68,NaN


In [25]:
# convert to lowercase
prison_filtered_copy['topic_description'] = prison_filtered_copy['topic_description'].str.lower()
# remove words that skew topic results
words_to_remove = ['new', 'york', 'times', 'author', 'novel']
prison_filtered_copy['topic_description'] = prison_filtered_copy['topic_description'].apply(
    lambda text: ' '.join([word for word in text.split() if word not in words_to_remove])
)

In [26]:
# create docs to be processed by BERTopic
docs = prison_filtered_copy.topic_description.values

In [21]:
# final settings for prison model
# compared all-MiniLM-L6-v2 and all-mpnet-base-v2 sentence transformers. Stuck with all-MiniLM-L6-v2 due to speed of training and more accurate representation
embedding_model = SentenceTransformer("all-MiniLM-L6-v2") 

umap_model = UMAP(n_neighbors=15, 
                  n_components=5, 
                  min_dist=0.0, 
                  metric='cosine')

hdbscan_model = HDBSCAN(min_cluster_size=15, 
                        metric='euclidean', 
                        cluster_selection_method='eom', 
                        prediction_data=True)

vectorizer_model = CountVectorizer(ngram_range=(1, 2), 
                                   stop_words="english")

ctfidf_model = ClassTfidfTransformer()

# increase word diversity in topic representation
representation_model = MaximalMarginalRelevance(diversity=0.5)

# put all steps together for final model
topic_model = BERTopic(
  embedding_model=embedding_model,    
  umap_model=umap_model,              
  hdbscan_model=hdbscan_model,        
  vectorizer_model=vectorizer_model,  
  ctfidf_model=ctfidf_model,          
  representation_model=representation_model,                      
  calculate_probabilities=True,        
  verbose=True
)

topics, probs = topic_model.fit_transform(docs)
topic_model.get_topic_info()

Batches:   0%|          | 0/368 [00:00<?, ?it/s]

2023-06-02 01:58:42,836 - BERTopic - Transformed documents to Embeddings
2023-06-02 01:58:54,003 - BERTopic - Reduced dimensionality
2023-06-02 01:59:00,922 - BERTopic - Clustered reduced embeddings


,Topic,Count,Name,Representation,Representative_Docs
0,-1,4890,-1_book_man_family_series,"[book, man, family, series, woman, bestselling...","[bestselling life after death, the hip-hop gen..."
1,0,575,0_erotica_women_pleasure_sexuality,"[erotica, women, pleasure, sexuality, position...",[if you want to learn how to give a woman oral...
2,1,357,1_white_history_african american_slavery,"[white, history, african american, slavery, mo...",[winner of the 2018 naacp image award for outs...
3,2,292,2_viz media_demon_ichigo_manga,"[viz media, demon, ichigo, manga, asako, stude...",[fight hellfire with hellfire! reads r to l (j...
4,3,251,3_vampire_bestselling_series_warrior,"[vampire, bestselling, series, warrior, fantas...",[#1 bestselling nora roberts presents the elec...
...,...,...,...,...,...
102,101,16,101_quilts_pattern design_friendship bracelets...,"[quilts, pattern design, friendship bracelets,...","[quilting is a hugely popular pastime, with ma..."
103,102,16,102_gadgets_evil genius_telephone_graphene,"[gadgets, evil genius, telephone, graphene, sp...",[engage your warped sense of humor with hundre...
104,103,16,103_julius caesar_hannibal_empire_romans,"[julius caesar, hannibal, empire, romans, cart...",[from the of the bestselling the dangerous boo...
105,104,16,104_spectrum series_artists_year review_invalu...,"[spectrum series, artists, year review, invalu...","[challenging, controversial, educational, and ..."


In [28]:
topic_model.get_topic_info().head(50)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4890,-1_book_man_family_series,"[book, man, family, series, woman, bestselling...","[bestselling life after death, the hip-hop gen..."
1,0,575,0_erotica_women_pleasure_sexuality,"[erotica, women, pleasure, sexuality, position...",[if you want to learn how to give a woman oral...
2,1,357,1_white_history_african american_slavery,"[white, history, african american, slavery, mo...",[winner of the 2018 naacp image award for outs...
3,2,292,2_viz media_demon_ichigo_manga,"[viz media, demon, ichigo, manga, asako, stude...",[fight hellfire with hellfire! reads r to l (j...
4,3,251,3_vampire_bestselling_series_warrior,"[vampire, bestselling, series, warrior, fantas...",[#1 bestselling nora roberts presents the elec...
5,4,209,4_witchcraft_rituals_spells_angels,"[witchcraft, rituals, spells, angels, occult, ...",[a practical guide to dr. john dee’s angelic m...
6,5,190,5_eminem_tupac_rap_interviews,"[eminem, tupac, rap, interviews, story, manson...","[it has been reviled, dismissed, attacked, and..."
7,6,183,6_real estate_investment_marketing_start,"[real estate, investment, marketing, start, sm...",[do you want to learn how to make day trading ...
8,7,178,7_designs_tattoo artists_body art_tattoo designs,"[designs, tattoo artists, body art, tattoo des...",[a groundbreaking reference covering the key s...
9,8,152,8_serial killers_true crime_cases_murders,"[serial killers, true crime, cases, murders, g...",[the killer book of serial killers is the ulti...


In [ ]:
# save model without embedding to save space (you can load it from the cloud)
topic_model.save("prison_model_mini", save_embedding_model=False)

In [10]:
# load saved model
topic_model=BERTopic.load("prison_model_mini", embedding_model="all-MiniLM-L6-v2")

In [5]:
topic_model.visualize_topics()

In [12]:
topic_model.visualize_barchart(top_n_topics=12, n_words=8)

In [13]:
topic_model.visualize_hierarchy()

In [11]:
# reduce outliers using best term frequency score
topics = topic_model.topics_
new_topics = topic_model.reduce_outliers(docs, topics, strategy="c-tf-idf")

documents = pd.DataFrame({"Document": docs, "Topic": new_topics})
topic_model._update_topic_size(documents)
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,5,-1_book_man_family_series,"[book, man, family, series, woman, bestselling...","[bestselling life after death, the hip-hop gen..."
1,0,848,0_erotica_women_pleasure_sexuality,"[erotica, women, pleasure, sexuality, position...",[if you want to learn how to give a woman oral...
2,1,911,1_white_history_african american_slavery,"[white, history, african american, slavery, mo...",[winner of the 2018 naacp image award for outs...
3,2,415,2_viz media_demon_ichigo_manga,"[viz media, demon, ichigo, manga, asako, stude...",[fight hellfire with hellfire! reads r to l (j...
4,3,707,3_vampire_bestselling_series_warrior,"[vampire, bestselling, series, warrior, fantas...",[#1 bestselling nora roberts presents the elec...
...,...,...,...,...,...
102,101,19,101_quilts_pattern design_friendship bracelets...,"[quilts, pattern design, friendship bracelets,...","[quilting is a hugely popular pastime, with ma..."
103,102,21,102_gadgets_evil genius_telephone_graphene,"[gadgets, evil genius, telephone, graphene, sp...",[engage your warped sense of humor with hundre...
104,103,26,103_julius caesar_hannibal_empire_romans,"[julius caesar, hannibal, empire, romans, cart...",[from the of the bestselling the dangerous boo...
105,104,23,104_spectrum series_artists_year review_invalu...,"[spectrum series, artists, year review, invalu...","[challenging, controversial, educational, and ..."


In [12]:
topic_model.get_topic_info().head(50)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,5,-1_book_man_family_series,"[book, man, family, series, woman, bestselling...","[bestselling life after death, the hip-hop gen..."
1,0,848,0_erotica_women_pleasure_sexuality,"[erotica, women, pleasure, sexuality, position...",[if you want to learn how to give a woman oral...
2,1,911,1_white_history_african american_slavery,"[white, history, african american, slavery, mo...",[winner of the 2018 naacp image award for outs...
3,2,415,2_viz media_demon_ichigo_manga,"[viz media, demon, ichigo, manga, asako, stude...",[fight hellfire with hellfire! reads r to l (j...
4,3,707,3_vampire_bestselling_series_warrior,"[vampire, bestselling, series, warrior, fantas...",[#1 bestselling nora roberts presents the elec...
5,4,295,4_witchcraft_rituals_spells_angels,"[witchcraft, rituals, spells, angels, occult, ...",[a practical guide to dr. john dee’s angelic m...
6,5,300,5_eminem_tupac_rap_interviews,"[eminem, tupac, rap, interviews, story, manson...","[it has been reviled, dismissed, attacked, and..."
7,6,244,6_real estate_investment_marketing_start,"[real estate, investment, marketing, start, sm...",[do you want to learn how to make day trading ...
8,7,185,7_designs_tattoo artists_body art_tattoo designs,"[designs, tattoo artists, body art, tattoo des...",[a groundbreaking reference covering the key s...
9,8,335,8_serial killers_true crime_cases_murders,"[serial killers, true crime, cases, murders, g...",[the killer book of serial killers is the ulti...


In [27]:
# merge topics to corresponding descriptions in original dataframe and compare topics from previously ran model

# Step 1: Get topic information from the topic model and exclude outliers
topic_df = topic_model.get_topic_info()[1:]
#topic_df = topic_model.get_topic_info()

# Step 2: Create a DataFrame with descriptions and corresponding topics
topics_df = pd.DataFrame({"description": docs, "Topic": topic_model.topics_})

# Step 3: Merge topics DataFrame with topic information DataFrame
merged_df = topics_df.merge(topic_df, on='Topic', how='left')

# Step 4: Merge copied DataFrame with merged_df using concat
merged_s = pd.concat([prison_filtered_copy.reset_index(drop=True), merged_df['Name'].reset_index(drop=True)], axis=1)

# Step 5: Remove the number and underscore from the 'Name' column
merged_s['Name'] = merged_s['Name'].str.replace(r'^\d+_', '', regex=True)
merged_s

,Unnamed: 0,publication,author,date,year,month,day,reason,state_arc,publication_date,topic_description,author_2,publisher,printType,categories,State_Count,Topic,Name
0,3,[KOREAN LANGUAGE CHARACTERS].,NaN,2021-02-25,2021.0,2.0,25.0,NaN,FL,2021-02-10,are you studying korean and want to learn how ...,['Lilas Lingvo'],Independently Published,BOOK,NaN,20202,vocabulary_arabic_pronunciation_kanji,pronunciation_sign language_asl_audio
1,6,1 HOUR TRADE.,NaN,2019-07-31,2019.0,7.0,31.0,NaN,FL,2022-05-02T22:59:00Z,please note: this is a companion version & not...,"['Everest Media,']",Everest Media LLC,BOOK,['Business & Economics'],20202,stocks_investment_forex_traders,real estate_investment_marketing_start
2,10,10 WORST PSYCHOPATHS: THE MOST DEPRAVED KILLER...,NaN,2017-06-01,2017.0,6.0,1.0,NaN,FL,2016-10-15,"often, the term 'psychopath' tends to be equat...",['Victor McQueen'],Arcturus Editions,BOOK,['Murderers'],20202,serial_bundy_case_murders,serial killers_true crime_cases_murders
3,11,100 ANIMALS ADULT COLORING BOOK.,NaN,2019-08-28,2019.0,8.0,28.0,NaN,FL,2019-12-16,animal lovers coloring book with 100 gorgeous ...,['A Design Creation'],NaN,BOOK,NaN,20202,coloring_stress_designs_pages,coloring book_designs_pages_mandalas
4,13,100 BEST SOLITAIRE GAMES.,NaN,2013-07-17,2013.0,7.0,17.0,NaN,FL,2013-09-01,table { }.font5 { color: black; font-size: 8pt...,NaN,Cardoza Publishing,BOOK,['Games'],20202,poker_play_em_limit,poker_play_limit_font
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11758,55256,The Negro Question- Who am I,NaN,2021-04-29,2021.0,4.0,29.0,NaN,RI,2012-01-06,finally a book filled with archaeological evid...,['Lee Cummings'],NaN,BOOK,NaN,68,blacks_civil_rights_movement,white_history_african american_slavery
11759,55259,The Negro Question Part 4 - The Missing Link,NaN,2021-04-29,2021.0,4.0,29.0,NaN,RI,2015-03-05,this book is about the three black kingdoms of...,['Lee Cummings'],Createspace Independent Publishing Platform,BOOK,['Black people'],68,blacks_civil_rights_movement,white_history_african american_slavery
11760,55262,Evil Women Deadlier than the male,NaN,2021-07-01,2021.0,7.0,1.0,NaN,RI,2012-07-13,among the thirty-four cases covered are: pamel...,['John Marlowe'],Arcturus Publishing,BOOK,['True Crime'],68,nursing_prescription_monographs_interactions,el chapo_cartels_dea_narco
11761,55271,Fentanyl,NaN,2021-08-26,2021.0,8.0,26.0,NaN,RI,2020-03-14,is there a way to end north america’s opioid e...,['Brodie Ramin'],Dundurn,BOOK,['Medical'],68,clinical_neurology_practice_adhd,information_nursing_symptoms_medications


In [30]:
# rename column in merged_s and prison_filtered dataframe for merging
merged_s = merged_s.rename(columns={'topic_description': 'topic_description_lowercase'})
prison_filtered['topic_description_lowercase'] = prison_filtered['topic_description'].str.lower()

# remove same words from the original prison_filtered df for merging on the topic_description
words_to_remove = ['new', 'york', 'times', 'author', 'novel']

# remove specified words from topic_description_lowercase column in prison_filtered dataframe
prison_filtered['topic_description_lowercase'] = prison_filtered['topic_description_lowercase'].apply(
    lambda text: ' '.join([word for word in text.split() if word not in words_to_remove])
)

# merge prison_filtered and merged_s dataframes based on topic_description_lowercase column
merged_df1 = prison_filtered.merge(merged_s[['topic_description_lowercase', 'Name']],
                                   on='topic_description_lowercase',
                                   how='left')

new_df = merged_df1.drop_duplicates()

new_df = new_df.drop("topic_description_lowercase", axis=1)

# concatenate the prison dataset which had missing topic descriptions with the df that had topic descriptions and corresponding topics
concatenated_df = pd.concat([df_with_nan, new_df], axis=0, ignore_index=True)

# sort the concatenated dataframe in order to match the order of original dataset
concatenated_df = concatenated_df.sort_values(by='Unnamed: 0').reset_index(drop=True)

concatenated_df = concatenated_df.drop("Unnamed: 0", axis=1)

concatenated_df


,publication,author,date,year,month,day,reason,state_arc,publication_date,topic_description,author_2,publisher,printType,categories,State_Count,Topic,Name
0,[ARABIC LANGUAGE CHARACTERS].,NaN,2019-12-18,2019.0,12.0,18.0,NaN,FL,1806,NaN,['Ahmad ibn Abou Bakr ibn Wahschih'],NaN,BOOK,NaN,20202,NaN,NaN
1,[ARABIC LANGUAGE CHARACTERS]. TAJWEED QURAN.,NaN,2021-09-09,2021.0,9.0,9.0,NaN,FL,NaN,NaN,NaN,NaN,NaN,NaN,20202,NaN,NaN
2,[KOREAN LANGUAGE CHARACTERS] HOLY SPIRIT LOVE.,NaN,2021-02-25,2021.0,2.0,25.0,NaN,FL,NaN,NaN,NaN,NaN,NaN,NaN,20202,NaN,NaN
3,[KOREAN LANGUAGE CHARACTERS].,NaN,2021-02-25,2021.0,2.0,25.0,NaN,FL,2021-02-10,Are you studying Korean and want to learn how ...,['Lilas Lingvo'],Independently Published,BOOK,NaN,20202,vocabulary_arabic_pronunciation_kanji,pronunciation_sign language_asl_audio
4,[KOREAN LANGUAGE CHARACTERS].,NaN,2021-02-25,2021.0,2.0,25.0,NaN,FL,2021-02-10,Are you studying Korean and want to learn how ...,['Lilas Lingvo'],Independently Published,BOOK,NaN,20202,vocabulary_arabic_pronunciation_kanji,pronunciation_sign language_asl_audio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55273,Mentalism Companion,NaN,2021-08-25,2021.0,8.0,25.0,NaN,RI,NaN,NaN,NaN,NaN,NaN,NaN,68,NaN,NaN
55274,"You Can Read Anyone, Rules of the Game",NaN,2021-09-22,2021.0,9.0,22.0,NaN,RI,NaN,NaN,NaN,NaN,NaN,NaN,68,NaN,NaN
55275,Sex Slave never judge a Women by her desires,NaN,2021-09-30,2021.0,9.0,30.0,NaN,RI,NaN,NaN,NaN,NaN,NaN,NaN,68,NaN,NaN
55276,Moorish American Civics Guide,NaN,2021-09-30,2021.0,9.0,30.0,NaN,RI,2014-12-22,This civics guide is designed for those who la...,['Sheik Way-El'],Lulu.com,BOOK,NaN,68,blacks_civil_rights_movement,white_history_african american_slavery


In [41]:
# drop rows with NaN values in 'date' column in order for time series visualization
no_dates = merged_s.dropna(subset=['date']).copy()
no_dates

,Unnamed: 0,publication,author,date,year,month,day,reason,state_arc,publication_date,topic_description_lowercase,author_2,publisher,printType,categories,State_Count,Topic,Name
0,3,[KOREAN LANGUAGE CHARACTERS].,NaN,2021-02-25,2021.0,2.0,25.0,NaN,FL,2021-02-10,are you studying korean and want to learn how ...,['Lilas Lingvo'],Independently Published,BOOK,NaN,20202,vocabulary_arabic_pronunciation_kanji,pronunciation_sign language_asl_audio
1,6,1 HOUR TRADE.,NaN,2019-07-31,2019.0,7.0,31.0,NaN,FL,2022-05-02T22:59:00Z,please note: this is a companion version & not...,"['Everest Media,']",Everest Media LLC,BOOK,['Business & Economics'],20202,stocks_investment_forex_traders,real estate_investment_marketing_start
2,10,10 WORST PSYCHOPATHS: THE MOST DEPRAVED KILLER...,NaN,2017-06-01,2017.0,6.0,1.0,NaN,FL,2016-10-15,"often, the term 'psychopath' tends to be equat...",['Victor McQueen'],Arcturus Editions,BOOK,['Murderers'],20202,serial_bundy_case_murders,serial killers_true crime_cases_murders
3,11,100 ANIMALS ADULT COLORING BOOK.,NaN,2019-08-28,2019.0,8.0,28.0,NaN,FL,2019-12-16,animal lovers coloring book with 100 gorgeous ...,['A Design Creation'],NaN,BOOK,NaN,20202,coloring_stress_designs_pages,coloring book_designs_pages_mandalas
4,13,100 BEST SOLITAIRE GAMES.,NaN,2013-07-17,2013.0,7.0,17.0,NaN,FL,2013-09-01,table { }.font5 { color: black; font-size: 8pt...,NaN,Cardoza Publishing,BOOK,['Games'],20202,poker_play_em_limit,poker_play_limit_font
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11758,55256,The Negro Question- Who am I,NaN,2021-04-29,2021.0,4.0,29.0,NaN,RI,2012-01-06,finally a book filled with archaeological evid...,['Lee Cummings'],NaN,BOOK,NaN,68,blacks_civil_rights_movement,white_history_african american_slavery
11759,55259,The Negro Question Part 4 - The Missing Link,NaN,2021-04-29,2021.0,4.0,29.0,NaN,RI,2015-03-05,this book is about the three black kingdoms of...,['Lee Cummings'],Createspace Independent Publishing Platform,BOOK,['Black people'],68,blacks_civil_rights_movement,white_history_african american_slavery
11760,55262,Evil Women Deadlier than the male,NaN,2021-07-01,2021.0,7.0,1.0,NaN,RI,2012-07-13,among the thirty-four cases covered are: pamel...,['John Marlowe'],Arcturus Publishing,BOOK,['True Crime'],68,nursing_prescription_monographs_interactions,el chapo_cartels_dea_narco
11761,55271,Fentanyl,NaN,2021-08-26,2021.0,8.0,26.0,NaN,RI,2020-03-14,is there a way to end north america’s opioid e...,['Brodie Ramin'],Dundurn,BOOK,['Medical'],68,clinical_neurology_practice_adhd,information_nursing_symptoms_medications


In [42]:
dated_docs = no_dates.topic_description_lowercase.values

In [43]:
# running separate model where all dates were included in order to visualize topics over time
embedding_model = SentenceTransformer("all-MiniLM-L6-v2") 

umap_model = UMAP(n_neighbors=15, 
                  n_components=5, 
                  min_dist=0.0, 
                  metric='cosine')

hdbscan_model = HDBSCAN(min_cluster_size=15, 
                        metric='euclidean', 
                        cluster_selection_method='eom', 
                        prediction_data=True)

vectorizer_model = CountVectorizer(ngram_range=(1, 2), 
                                   stop_words="english")

ctfidf_model = ClassTfidfTransformer()

# increase word diversity in topic representation
representation_model = MaximalMarginalRelevance(diversity=0.5)

# put all steps together for final model
topic_model_dates = BERTopic(
  embedding_model=embedding_model,    
  umap_model=umap_model,              
  hdbscan_model=hdbscan_model,        
  vectorizer_model=vectorizer_model,  
  ctfidf_model=ctfidf_model,          
  representation_model=representation_model,                      
  calculate_probabilities=True,        
  verbose=True
)

topics, probs = topic_model_dates.fit_transform(dated_docs)
topic_model_dates.get_topic_info()

Batches:   0%|          | 0/314 [00:00<?, ?it/s]

2023-06-04 17:17:45,350 - BERTopic - Transformed documents to Embeddings
2023-06-04 17:18:18,164 - BERTopic - Reduced dimensionality
2023-06-04 17:18:22,045 - BERTopic - Clustered reduced embeddings


,Topic,Count,Name,Representation,Representative_Docs
0,-1,4186,-1_book_man_family_series,"[book, man, family, series, art, american, wom...","[""the right story at the right time. . . . it’..."
1,0,605,0_women_pleasure_sexuality_erotica,"[women, pleasure, sexuality, erotica, position...",[legendary porn star nina hartley's guide to t...
2,1,438,1_life_man_streets_woman,"[life, man, streets, woman, heart, father, dra...",[whether it's being told you have to purchase ...
3,2,240,2_vampire_series_bestselling_fantasy,"[vampire, series, bestselling, fantasy, warrio...",[***over a million copies of the iron druid bo...
4,3,227,3_viz media_ichigo_battle_tokyo,"[viz media, ichigo, battle, tokyo, manga, acad...","[in an office romance, there's a fine line bet..."
...,...,...,...,...,...
88,87,17,87_vietnam war_soldier_fury_trenches,"[vietnam war, soldier, fury, trenches, corbett...",[the complete text of the bestselling narrativ...
89,88,16,88_fantasy_zeno_artists_com daniel,"[fantasy, zeno, artists, com daniel, godmachin...","[since his recent arrival, avant-garde artist ..."
90,89,16,89_clint adams_million gunsmith_books print_cash,"[clint adams, million gunsmith, books print, c...",[business and pleasure clint adams is just as ...
91,90,16,90_patrice_southwest_maroons_cherokee,"[patrice, southwest, maroons, cherokee, columb...",[for four hundred years--from the first spanis...


In [44]:
# reduce outliers using best term frequency score
topics = topic_model_dates.topics_
new_topics = topic_model_dates.reduce_outliers(dated_docs, topics, strategy="c-tf-idf")

documents = pd.DataFrame({"Document": dated_docs, "Topic": new_topics})
topic_model_dates._update_topic_size(documents)
topic_model_dates.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,9,-1_book_man_family_series,"[book, man, family, series, art, american, wom...","[""the right story at the right time. . . . it’..."
1,0,775,0_women_pleasure_sexuality_erotica,"[women, pleasure, sexuality, erotica, position...",[legendary porn star nina hartley's guide to t...
2,1,1418,1_life_man_streets_woman,"[life, man, streets, woman, heart, father, dra...",[whether it's being told you have to purchase ...
3,2,432,2_vampire_series_bestselling_fantasy,"[vampire, series, bestselling, fantasy, warrio...",[***over a million copies of the iron druid bo...
4,3,265,3_viz media_ichigo_battle_tokyo,"[viz media, ichigo, battle, tokyo, manga, acad...","[in an office romance, there's a fine line bet..."
...,...,...,...,...,...
88,87,25,87_vietnam war_soldier_fury_trenches,"[vietnam war, soldier, fury, trenches, corbett...",[the complete text of the bestselling narrativ...
89,88,38,88_fantasy_zeno_artists_com daniel,"[fantasy, zeno, artists, com daniel, godmachin...","[since his recent arrival, avant-garde artist ..."
90,89,22,89_clint adams_million gunsmith_books print_cash,"[clint adams, million gunsmith, books print, c...",[business and pleasure clint adams is just as ...
91,90,25,90_patrice_southwest_maroons_cherokee,"[patrice, southwest, maroons, cherokee, columb...",[for four hundred years--from the first spanis...


In [53]:
topic_model_dates.get_topic_info().head(50)

,Topic,Count,Name,CustomName,Representation,Representative_Docs
0,-1,9,-1_book_man_family_series,"book, man, family, series","[book, man, family, series, art, american, wom...","[""the right story at the right time. . . . it’..."
1,0,775,0_women_pleasure_sexuality_erotica,"women, pleasure, sexuality, erotica","[women, pleasure, sexuality, erotica, position...",[legendary porn star nina hartley's guide to t...
2,1,1418,1_life_man_streets_woman,"life, man, streets, woman","[life, man, streets, woman, heart, father, dra...",[whether it's being told you have to purchase ...
3,2,432,2_vampire_series_bestselling_fantasy,"vampire, series, bestselling, fantasy","[vampire, series, bestselling, fantasy, warrio...",[***over a million copies of the iron druid bo...
4,3,265,3_viz media_ichigo_battle_tokyo,"viz media, ichigo, battle, tokyo","[viz media, ichigo, battle, tokyo, manga, acad...","[in an office romance, there's a fine line bet..."
5,4,284,4_witchcraft_rituals_spells_occult,"witchcraft, rituals, spells, occult","[witchcraft, rituals, spells, occult, grimoire...","[a collector’s edition of the classic, illustr..."
6,5,249,5_hip hop_punk_eminem_tupac,"hip hop, punk, eminem, tupac","[hip hop, punk, eminem, tupac, interviews, sto...",[tracing the careers of hip-hop's three most d...
7,6,171,6_artists_ink_tattoo artist_body art,"artists, ink, tattoo artist, body art","[artists, ink, tattoo artist, body art, tattoo...","[learn how to create bold, original designs fr..."
8,7,362,7_white_history_movement_power,"white, history, movement, power","[white, history, movement, power, civil rights...",[an intersectional history of the shared strug...
9,8,404,8_detective_thriller_crime_bestselling,"detective, thriller, crime, bestselling","[detective, thriller, crime, bestselling, murd...",[bestseller • features a introduction on the o...


In [47]:
# Prepare timestamps
timestamps = no_dates['date'].to_list()

# Create topics over time
topics_over_time = topic_model_dates.topics_over_time(
    dated_docs, 
    timestamps,
    nr_bins=20, 
    global_tuning=True, 
    evolution_tuning=True
)

# Customize topic labels
topic_labels = topic_model_dates.generate_topic_labels(nr_words=4,
                                                 topic_prefix=False,
                                                 separator=", ")

topic_model_dates.set_topic_labels(topic_labels)

20it [02:07,  6.38s/it]


In [63]:
import plotly.express as px
import plotly.io as pio

# Choose some interesting topics to visualize over time
fig = topic_model_dates.visualize_topics_over_time(topics_over_time, topics=[3,5,9,11,32], custom_labels=True)

# Move legend location
fig.update_layout(
    legend=dict(
        yanchor="top",
        y=1,
        xanchor="left",
        x=0.01
    )
)

# Update titles and axes
fig.update_layout(
    title={
        'text': "Banned Prison Topics Over Time",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title="Year Banning Took Place",
    yaxis_title="Frequency",
    font=dict(
        family="Arial",
        size=14,
        color="black"
    ),
    margin=dict(
        l=50,
        r=50,
        t=80,
        b=50
    ),
    plot_bgcolor="#f2f2f2",
    paper_bgcolor="#f2f2f2",
    autosize=False,
    width=800,
    height=500
)

fig.show()

In [65]:
pio.write_html(fig, file='prison_topics_over_time.html')


In [16]:
# compare generated topics from previous model
concatenated_df['Topic'].value_counts().head(20)

romance_man_wants_bestselling               1505
gangsta_blood_crew_street                    544
blacks_civil_rights_movement                 509
detective_sara_thriller_jeffrey              450
tupac_rap_interviews_manson                  446
oral_positions_partner_orgasms               442
erotica_threesome_women_anthology            410
tattoos_designs_artists_coloring             397
serial_bundy_case_murders                    367
magic_shannara_brooks_druid                  332
witchcraft_rituals_spells_grimoire           322
nude_lingerie_beauty_rana                    321
copyright_books_culturally_imperfections     307
paintings_boris_palaces_work                 295
law_inmate_prisons_incarceration             290
persuasion_psychology_laws_communication     278
asako_llama_battle_haru                      258
king_stephen_series_tales                    256
survival_skills_shelter_guide                244
martial_jitsu_arts_aikido                    236
Name: Topic, dtype: 

In [17]:
# this representation is more balanced and accurate upon investigating
concatenated_df['Name'].value_counts().head(20)

erotica_women_pleasure_sexuality                                       1392
white_history_african american_slavery                                 1156
vampire_bestselling_series_warrior                                      852
gangsta_blood_enemies_street                                            713
romance_bestselling_cowboy_doesn                                        656
viz media_demon_ichigo_manga                                            517
eminem_tupac_rap_interviews                                             486
serial killers_true crime_cases_murders                                 405
designs_tattoo artists_body art_tattoo designs                          398
maggie_detective_murder_sara                                            386
witchcraft_rituals_spells_angels                                        374
paintings_fantasy_work_collection                                       342
models_magazines_erotic_rana                                            313
real estate_

In [31]:
concatenated_df.to_csv('prison_without_index2.csv', index=False)